In [4]:
%matplotlib inline
import pandas as pd
import numpy as np 
from pandas import DataFrame
import matplotlib.pyplot as plt
from collections import Counter

In [1]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

FileNotFoundError: [Errno 2] No such file or directory: 'style-table.css'

In [136]:
#random Forest modeling
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error
def random_forests_regerssion(dataset, target):
    """ make predictions using Random Forest regression model. """

    estimators = range(100, 300, 20)
    new_MSE = 0.0
    prev_MSE = 99999.99
    
    #selecting best estimator
    for eID, estimator in enumerate(estimators):
        model = RandomForestRegressor(n_estimators=estimator,
                                      bootstrap=True,
                                      oob_score=True,
                                      n_jobs=-1 ) 
        regr = model.fit(dataset, target)

        f_importances = regr.feature_importances_
        expected  = target
        
        predicted = regr.predict(dataset)
        COD = regr.score(dataset , target)
        new_MSE = mean_squared_error(predicted, expected)

        if new_MSE < prev_MSE :
            prev_MSE = best_MSE = new_MSE
            best_COD = COD

            best_features =  f_importances
            best_estimator = estimator

            imp_feat_ids = [(imp_id, imp_val) for imp_id, imp_val in enumerate (best_features)]
            imp_feat_ids.sort(key=lambda tup: tup[1], reverse=True)

            top_five_imp_feat = [item[0] for item in imp_feat_ids[:5]] 
            
    best_features = np.asarray(best_features)
    
    #reduced_feat = [imp_id+1 for imp_id, imp_val in enumerate (best_features) if imp_val < np.mean(best_features)]
    print ("RF: ",best_COD, best_estimator, imp_feat_ids[:5])

    return top_five_imp_feat

In [137]:
def copy_final_data(ydata_final_id):
    """ Copies the unique samples from master data file. selects one copy based in case of duplicates"""
    xdata_final = pd.DataFrame()
    ydata_final = pd.DataFrame()
    
    for idx, item in enumerate(ydata_final_id):
        new_data = pd.DataFrame(xdata[item:item+1].values, columns=xdata.columns)
        xdata_final = xdata_final.append(new_data)

    for idy, item in enumerate(ydata_final_id):
        new_data = pd.DataFrame(list(ydata[item:item+1].values), columns=ydata.columns)
        ydata_final = ydata_final.append(new_data) 
    
    return xdata_final, ydata_final

In [138]:
#data preprocessing
from sklearn.preprocessing import StandardScaler, RobustScaler

def data_scaling(X_train):
    """ Scaling  feature set"""

    # Scale data
    standard_scaler = StandardScaler()
    Xtr_s = standard_scaler.fit_transform(X_train)

    robust_scaler = RobustScaler(with_centering=False, copy=True, )
    Xtr_r = robust_scaler.fit_transform(X_train)

    return Xtr_s, Xtr_r

In [140]:
from  collections import OrderedDict
data_final_ids = dict()


def find_final_ids():
    """ removes the duplicate record in featur set, """
    
    global data_final_ids, xdata, ydata
    data_final_ids = dict()
    
    xdata_np = np.array(xdata.values)
    xdata_hashable =  list(map(tuple, xdata_np))
    xdata_set = set(xdata_hashable)

    data_final_ids = dict()
    losses = ['ten', 'twen', 'threefive', 'fifty', 'sixty', 'sevenfive','nine']

    for lid, loss in enumerate(losses):
        net_dict = dict()
        for idx, rec in enumerate(xdata_hashable):
            net_dict.setdefault(rec, []).append(idx)
  
        ydata_final_id = []
          
        for key in net_dict:
            if len(net_dict[key]) > 1:
                prr_select = {}
                for nidy in net_dict[key]:
                    prr_select.setdefault(ydata[loss][nidy],[]).append(nidy)
                
                ydata_nid_dict = OrderedDict(sorted(prr_select.items()))
                ydata_nid  = [(key, sorted(value)[0], len(value)) for (key, value) in prr_select.items()]
                ydata_nid.sort(key=lambda tup: tup[2], reverse=True)
                
                #append the least of all PRR values
                ydata_final_id.append(ydata_nid[0][1])
            else:
                ydata_final_id.append(net_dict[key][0])
                
        data_final_ids[loss] = ydata_final_id
        #print(len(data_final_ids[loss]))#, sorted(ydata_final_id))  
        

In [141]:
import numpy.ma as ma
def delete_feats(xdata_t, ydata_t):
    """deletes features with zero standerd deviation"""
    global xdata 
       
    matches = xdata_t.std(axis=0) > 0
    x_selected = xdata_t[matches[matches]]
    data_corr = np.corrcoef(x_selected)
    data_selec = ma.masked_where(data_corr < 0.95, data_corr)
    plt.pcolor(data_selec)
    plt.colorbar()
    plt.show()

In [ ]:
from  collections import OrderedDict
data_final_ids = dict()


def find_final_ids_pandas(xdata, ydata, loss):
    """ removes the duplicate record in featur set, """

    xdata_inter[loss] = ydata[loss]
    xdata_sorted = xdata_inter.sort([loss], ascending=False)
    cols = list(xdata_sorted.columns.values)[:-1]
    mask = xdata_sorted.duplicated(cols, keep='first')
    xdata_final = xdata_sorted[~mask]
    
    return xdata_final[~mask], xdata_final[loss]
        

In [ ]:
xdata = pd.DataFrame()
ydata = pd.DataFrame()

def load_data(net_size):
    """ loads the features(x) data and target(y_lable) data into dataframes"""
    global xdata, ydata
    
    #load x_data (feeture set)
    xdata = pd.DataFrame.from_csv('data/ecoli_%s_features.txt' %net_size, index_col=None)
    #xdata = pd.DataFrame.from_csv('../VMNs/data/ecoli-{0}-features.txt'.format(str(net_size)), index_col=None)
    #xdata.head()

    #load target data (y label)
    ydata = pd.DataFrame.from_csv('data/ecoli_%s_target.txt' %net_size, index_col=None)
    #ydata = pd.DataFrame.from_csv('../VMNs/data/ecoli_{0}_target.txt'.format(str(net_size)), index_col=None)
    #ydata.head()

In [142]:
def save_file(data_set, filen_name):
    #xfile_name = '../VMNs/data/ecoli_{0}_features_no_dup.txt'.format(net_size)
    data_Set.to_csv(file_name, sep='\t', encoding='utf-8')

In [ ]:
print('Hello World!!')

In [ ]:
losses = ['ten','twen', 'threefive', 'fifty', 'sixty', 'sevenfive', 'nine']
net_sizes = [100]#, 200, 300, 400, 500]


for net_size in net_sizes:
   
    load_data(str(net_size))
    print(net_size)
        
    '''
    for lid, loss in enumerate(losses):
        xdata_uni, ydata_uni = find_final_ids_pandas(xdata, ydata, loss)
        #xdata_final, ydata_final = copy_final_data(data_final_ids[loss])
        xdata_ss , xdata_rs = data_scaling(xadata_uni)
        #delete_feats(xdata_ss, ydata_final[loss])
                     
        top_five = random_forests_regerssion(xdata_ss, ydata_uni)
       
    print('--------------------------------------------------------------------------------------------')
    '''

In [143]:
losses = ['ten','twen', 'threefive', 'fifty', 'sixty', 'sevenfive', 'nine']
net_sizes = [100, 200, 300, 400, 500]


for net_size in net_sizes:
   
    load_data(str(net_size))
        
    for lid, loss in enumerate(losses):
        xdata_uni, ydata_uni = find_final_ids_pandas(xdata, ydata, loss)
        #xdata_final, ydata_final = copy_final_data(data_final_ids[loss])
        xdata_ss , xdata_rs = data_scaling(xadata_uni)
        #delete_feats(xdata_ss, ydata_final[loss])
                     
        top_five = random_forests_regerssion(xdata_ss, ydata_uni)
       
    print('------------------------------------------------------------------------------------------------')

RF:  0.88072336844 240 [(0, 0.21727604109643289), (7, 0.13645287298088579), (1, 0.089467106287371728), (13, 0.075060396265086454), (108, 0.052224032486163474)]
RF:  0.884175722822 280 [(0, 0.18544000780586509), (7, 0.14748399067205956), (1, 0.087336347752363874), (13, 0.077910038531908254), (2, 0.065227789425640059)]
RF:  0.881961768624 200 [(0, 0.15570009381345554), (7, 0.10757047037815928), (13, 0.10080404977424502), (176, 0.081720287430524674), (1, 0.079247784882198732)]
RF:  0.896557255145 220 [(61, 0.22933396149787302), (0, 0.088908812793228459), (2, 0.086675017015915998), (7, 0.079080544764304178), (108, 0.061503256024133694)]
RF:  0.896001515991 280 [(61, 0.25863103991778386), (0, 0.12090985762772323), (13, 0.10847722019412462), (1, 0.052717748416820705), (10, 0.041293080135368351)]
RF:  0.875610197281 260 [(61, 0.31380718362210347), (0, 0.08746330521985915), (13, 0.061419547958701852), (1, 0.04780747646384062), (19, 0.04007139393858862)]
RF:  0.939425195278 240 [(61, 0.38924023

In [63]:
xdata_mean = xdata.mean(axis=0).values
xdata_std = xdata.std(axis=0).values
xdata_var = xdata.var(axis=0).values
feat_mean_std = [(idx, xdata_mean[idx], xdata_std[idx], xdata_var[idx]) \
                 for idx in range( len(xdata.columns)) if xdata_mean[idx] > 0] 
feat_mean_std.sort(key=lambda tup: tup[1], reverse= True )


In [ ]:
Number of sample in each size after duplicate removal

EMNs
------------------------------------------------------------------------------------------------
Tuple
129 129
238 238
162 162
167 167
156 156
Frozenset
124 124
230 230
160 160
166 166
156 156
------------------------------------------------------------------------------------------------

VMNs
------------------------------------------------------------------------------------------------
Tuple
783 783
860 860
851 851
775 775
915 915
Frozenset
722 722
845 845
850 850
775 775
915 915
----------------------------------------------------------------------------------------------